## Logistic Regression

In [98]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Subset

limit = 60
# Load MNIST dataset
train_dataset = MNIST(root='../data', train=True, download=False, transform=transforms.ToTensor())
test_dataset  = MNIST(root='../data', train=False, download=False, transform=transforms.ToTensor())
train_dataset = Subset(train_dataset, range(limit))
test_dataset  = Subset(test_dataset, range(limit))

# Data loaders
mini_batch_train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
mini_batch_test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
import numpy as np
import torch.nn.functional as F

class LogisticRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(28 * 28, 10)
        nn.init.uniform_(self.linear.weight, -0.01, 0.01) 
        nn.init.zeros_(self.linear.bias)

    def softmax(self, x):
        exp_x = torch.exp(x - torch.max(x, dim=1, keepdim=True).values)
        return exp_x / exp_x.sum(dim=1, keepdim=True)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.linear(x)
        x = self.softmax(x)
        return x

def cross_entropy_loss(outputs, labels):
    log_probs = torch.log(outputs)
    loss = -torch.sum(labels * log_probs, dim=1)
    return torch.mean(loss)

num_classes = 10
epochs = 1
max_iter = 20
model = LogisticRegression()
criterion = nn.CrossEntropyLoss()
optimizer = optim.LBFGS(model.parameters(), max_iter=max_iter)

model.train()
for itr in range(max_iter):
    itr_loss = 0.0
    for i, (images, label) in enumerate(train_dataset):
        # print(f"Batch {i+1}/{len(train_dataset)}")    
        def loss_closure():
            optimizer.zero_grad()
            outputs = model(images)
            loss_val = criterion(outputs, torch.tensor([label]))
            loss_val.backward()
            return loss_val
        
        optimizer.step(loss_closure)

        # outputs = model(images)
        loss = loss_closure()
        itr_loss += loss.item()

        # print(f"Loss: {loss.item():.4f}")

    print("iteration = %4d   loss = %0.4f" % (itr, itr_loss))

iteration =    0   loss = 138.3624
iteration =    1   loss = 139.6690
iteration =    2   loss = 139.6690
iteration =    3   loss = 139.6690
iteration =    4   loss = 139.6690
iteration =    5   loss = 139.6690
iteration =    6   loss = 139.6690
iteration =    7   loss = 139.6690
iteration =    8   loss = 139.6690
iteration =    9   loss = 139.6690
iteration =   10   loss = 139.6690
iteration =   11   loss = 139.6690
iteration =   12   loss = 139.6690
iteration =   13   loss = 139.6690
iteration =   14   loss = 139.6690
iteration =   15   loss = 139.6690
iteration =   16   loss = 139.6690
iteration =   17   loss = 139.6690
iteration =   18   loss = 139.6690
iteration =   19   loss = 139.6690


In [61]:
# Print out all information in (model.parameters())
for name, param in model.named_parameters():
    print(f"Parameter Name: {name}")
    print(f"Parameter Shape: {param.shape}")
    print(f"Parameter Values: {param.data}")

Parameter Name: linear.weight
Parameter Shape: torch.Size([10, 784])
Parameter Values: tensor([[-0.0223,  0.0014, -0.0142,  ..., -0.0314,  0.0112, -0.0295],
        [ 0.0312,  0.0293,  0.0177,  ..., -0.0296, -0.0038, -0.0329],
        [-0.0283, -0.0137, -0.0010,  ...,  0.0102,  0.0261,  0.0173],
        ...,
        [ 0.0088,  0.0012,  0.0073,  ..., -0.0333,  0.0355, -0.0108],
        [ 0.0077, -0.0244,  0.0250,  ...,  0.0328, -0.0055, -0.0100],
        [ 0.0144, -0.0184,  0.0133,  ..., -0.0283, -0.0104, -0.0076]])
Parameter Name: linear.bias
Parameter Shape: torch.Size([10])
Parameter Values: tensor([-0.0173,  0.0190, -0.0232, -0.0220,  0.0068, -0.0030,  0.0239,  0.0156,
         0.0302,  0.0242])


In [100]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
print(f"Input: {input}")
target = torch.empty(3, dtype=torch.long).random_(5)
print(f"Target: {target}")
output = loss(input, target)
print(f"Output Loss: {output.item()}")

Input: tensor([[ 2.8763,  0.0279, -0.4859,  0.3026,  0.8348],
        [ 1.2505, -0.1971, -0.6957,  1.0116, -1.1143],
        [-0.8433, -1.8453,  0.2537, -0.2584,  0.7560]], requires_grad=True)
Target: tensor([2, 1, 2])
Output Loss: 2.398954153060913


In [101]:
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
print(f"Input: {input}")
target = torch.randn(3, 5).softmax(dim=1)
print(f"Target: {target}")
output = loss(input, target)
print(f"Output Loss: {output.item()}")

Input: tensor([[ 0.4588,  1.6342,  0.2501, -1.0978, -0.7803],
        [ 0.0698, -0.6374,  1.7318, -0.1972, -0.9900],
        [-1.5758, -1.8658,  2.0320, -0.4963, -0.4279]], requires_grad=True)
Target: tensor([[0.3987, 0.1619, 0.0975, 0.3067, 0.0351],
        [0.0325, 0.0933, 0.4083, 0.2549, 0.2110],
        [0.7586, 0.0757, 0.1208, 0.0358, 0.0092]])
Output Loss: 2.3818016052246094
